- this file we generate input for idcf algorithm for douban bookreview dataset. 
- it generates core and non core users
- core users goes as an input to idcf
- non core user goes as an input to GMF
- We generate non core user using idcf algorithm
- Using generated non core user we tweak the original non core user and generate tweaked matrix
- We give tweaked matrix as an input to GMF
- we compare the results.

In [1]:
from fileinput import filename
import pandas as pd
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import svds
import random 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import scipy.stats as ss
import pickle

- code to read and clean douban book review file.

In [2]:
ml1m_dir = 'data/bookreviews_cleaned.txt'
# ml1m_rating = pd.read_csv(ml1m_dir, delim_whitespace=True,index_col=0, engine='python')
ml1m_rating = pd.read_csv(ml1m_dir, delim_whitespace=True, usecols=range(3), header=None)
# df = ml1m_rating[["user_id", book_id", "rating", "time"]]
ml1m_rating.head()
ml1m_rating.to_csv('data/douban_bookreview.csv', index=False)


In [2]:
def ReadData(threshold=30):
    # uncomment this line for movielens dataset
    ml1m_dir = 'data/douban_bookreview.csv'
    ml1m_rating = pd.read_csv(ml1m_dir, sep=',', header=None, names=['uid', 'mid', 'rating'],  engine='python')
    # use the below 2 lines for pinterest dataset.
    # ml1m_dir = 'data/pin-interest-main.txt'
    # ml1m_rating = pd.read_csv(ml1m_dir, delim_whitespace=True, header=None, names=['uid', 'mid', 'rating', 'timestamp'],  engine='python')

    unique_uid = np.unique(np.array(ml1m_rating['uid'].tolist()))
    unique_mid = np.unique(np.array(ml1m_rating['mid'].tolist()))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    mid_dict = dict([(y,x) for x,y in enumerate(unique_mid)])
    print('DICTIONARY PREPARED:')

    # init user item dictionary:
    
    uid_list = ml1m_rating['uid'].tolist()
    uid_list_len = len(uid_list)
    mid_list = ml1m_rating['mid'].tolist()
    mid_list_len = len(mid_list)
    rating_list = ml1m_rating['rating'].tolist()
    user_item_dict = {x:set() for x in range(len(unique_uid))}
    item_user_dict = {x:set() for x in range(len(unique_mid))}
    for i in range(uid_list_len):
        uid_list[i] = uid_dict[uid_list[i]]
        mid_list[i] = mid_dict[mid_list[i]]
        # rating_list[i] = 1 # comment this line if you want to activate explicit ratings
        user_item_dict[uid_list[i]].add(mid_list[i])
        item_user_dict[mid_list[i]].add(uid_list[i])
    tmp_df = pd.DataFrame({"uid":uid_list, "mid":mid_list, "ratings":rating_list})
    v = tmp_df.uid.value_counts()
    tmp_df = pd.DataFrame({"uid":uid_list, "mid":mid_list, "ratings":rating_list})
    # v = tmp_df.uid.value_counts()
    # df = tmp_df[tmp_df.uid.isin(v.index[v.gt(30)])]
### code to store less than 30 interactions:
    # df_less_30 = tmp_df[tmp_df.uid.isin(v.index[v.le(30)])]
    return tmp_df, len(np.unique(mid_list)), len(unique_uid)

In [3]:
df, unique_mids, unique_uids = ReadData()
print("UNIQUE MIDS: ", unique_mids)
print("UNIQUE UIDS: ", unique_uids)

DICTIONARY PREPARED:
UNIQUE MIDS:  95872
UNIQUE UIDS:  2212


In [4]:
# support_test_df = df_gt_30.groupby("uid").tail(1)
# # print(len(df_gt_30['uid']))
# support_train_df = df_gt_30.drop(df_gt_30.groupby('uid').tail(1).index, inplace=False)
# assert(len(df_gt_30)== len(support_test_df) + len(support_train_df))
# # print(len(test_df))
# # print(len(train_df))
# query_test_df = df_le_30.groupby("uid").tail(1)
# query_train_df = df_le_30.drop(df_le_30.groupby('uid').tail(1).index, inplace=False)
# assert(len(df_le_30)== len(query_test_df) + len(query_train_df))
# dic_support_train_df_uid_mapping = dict([(y,x) for x,y in enumerate(np.unique(support_train_df['uid']))])
# dic_support_train_df_uid_rmapping = dict([(x,y) for x,y in enumerate(np.unique(support_train_df['uid']))])
# ### no need for mid mapping

# uid_of_train_df = support_train_df['uid'].tolist()
# for i in range(len(uid_of_train_df)):
#     uid_of_train_df[i] = dic_support_train_df_uid_mapping[uid_of_train_df[i]]
# # for index, row in train_df.iterrows():
# #     train_df['uid'][index] = dic_train_df_uid_mapping[train_df['uid'][index]]
# core_user_ko_input_train_df = pd.DataFrame({'uid':uid_of_train_df, 'mid':support_train_df['mid'], 'ratings':support_train_df['ratings']})
# print("NUMBER OF USERS IN SUPPORT TEST:", len(np.unique(support_test_df['uid'])))
# print("NUMBER OF USERS IN SUPPORT TRAIN", len(np.unique(support_train_df['uid'])))
# print("NUMBER OF USERS IN QUERY TEST:", len(np.unique(query_test_df['uid'])))
# print("NUMBER OF USERS IN QUERY TRAIN", len(np.unique(query_train_df['uid'])))

NUMBER OF USERS IN SUPPORT TEST: 1371
NUMBER OF USERS IN SUPPORT TRAIN 1371
NUMBER OF USERS IN QUERY TEST: 841
NUMBER OF USERS IN QUERY TRAIN 725


In [5]:
# train_ui_dic = {}    
# for user in range(unique_uids):
#     train_ui_dic[user] = []
# for index,row in support_train_df.iterrows():
#         train_ui_dic[row['uid']].append(row['mid'])

- utility functions for CUR coreusers.

In [4]:
MAX_MID = 27277 + 1
def select_cols(mat, k, dup=False):
    # prob 1d array of probabilities of all columns
    prob = mat.T.dot(mat)
    prob = np.array(np.diagonal(prob))
    denom = np.abs(prob).sum(axis = 0)
    prob = prob/denom

    C = np.zeros((mat.shape[0], k))
    ind_cols = np.arange(0, prob.size)
    c_ind = []
    i = 0
    while(i < k):
        rand_sel = np.random.choice(ind_cols, 1, p=prob)
        if rand_sel in c_ind:
            continue
        c_ind.append(rand_sel[0])
        C[:, i] = mat[:, rand_sel[0]]
        i += 1
        # C[:, i] = C[:, i]/np.sqrt(k*prob[rand_sel[0]])

    return C, c_ind

def select_rows(mat, k, dup=False):

    prob = mat.dot(mat.T)
    prob = np.array(np.diagonal(prob))
    denom = np.abs(prob).sum(axis=0)
    prob = prob/denom
    print(prob)
    r = np.zeros((k, mat.shape[1]))
    ind_rows = np.arange(0, prob.size)
    r_ind = []
    i = 0
    while(i < k):
        # print(ind_rows)
        rand_sel = np.random.choice(ind_rows, 1, p=prob)
        if rand_sel in r_ind:
            continue
        r_ind.append(rand_sel[0])
        r[i, :] = mat[rand_sel[0], :]
        i += 1
        # r[i, :] = r[i, :]/np.sqrt(k*prob[rand_sel[0]])
    r_ind = np.array(r_ind)
    return r, r_ind

# def matIntersection(mat, c_ind, r_ind):
    
#     W = np.zeros((len(r_ind), len(c_ind)))
#     for i in range(len(r_ind)):
#         W[i] = mat[r_ind[i], c_ind]
    
#     return W

# def pseudoInverse(W):
#     # U = WP (W+)

#     # W = X.Z.YT
#     X, Z, YT = np.linalg.svd(W)
    
#     # W+ = Y.Z+.XT
#     XT = X.T
#     Y = YT.T
#     # Z+ = reciprocal(Z)
#     ZP = np.reciprocal(Z)
#     ZP = sp.spdiags(ZP, 0, ZP.size, ZP.size)
#     ZP = ZP@ZP
    
#     # W+ = Y.Z+.XT
#     WP = Y@ZP
#     WP = WP@XT

#     return WP

In [5]:
def ExtractCoreUsers(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    userItemMatrix = np.zeros(shape=(u_len, m_len))
    # print(userItemMatrix)
    
    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])
    
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[int(row['uid'])]][int(row['mid'])] = int(row['ratings'])
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    df = pd.DataFrame(userItemMatrix)
    cosineSimilarity = cosine_similarity(df)
    print("SHAPE OF COSINE MATIX:\n ", cosineSimilarity.shape)

    listToStoreTopFiftyOfEveryUser = []
    for i in range(0, cosineSimilarity.shape[0]):
        idx = np.argpartition(cosineSimilarity[i], -50)[-50:]
        listToStoreTopFiftyOfEveryUser.append(idx)
    # print("Top fifty list: \n", listToStoreTopFiftyOfEveryUser)
    # listToStoreTopFiftyOfEveryUser = np.array(listToStoreTopFiftyOfEveryUser)
    flatten = np.concatenate(listToStoreTopFiftyOfEveryUser)
    listToStoreTopFiftyOfEveryUser = flatten.ravel()

    # print("List of top 50", listToStoreTopFiftyOfEveryUser)
    df = pd.DataFrame(listToStoreTopFiftyOfEveryUser)
    allUserList = df.value_counts().index.tolist()
    # print("ALL USERS LIST", allUserList)
    allUserList = list(sum(allUserList,()))
    # print("ALL USERS LIST", allUserList)
    twentyPercentUserList = allUserList[:int(len(allUserList)*0.2)]
    # print("TWENTY PERCENT USER:", len(twentyPercentUserList))
    # print("TWENTY PERCENT USER:", (twentyPercentUserList))
    
    r_ind = twentyPercentUserList
    len_r_ind = len(r_ind)
    new_r_ind = [rev_uid_dict[r_ind[i]] for i in range(len_r_ind)]
    cos_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(new_r_ind))]
    # coreusers = dataframe.iloc[np.where(dataframe.uid.isin(twentyPercentUserList))]
    # coreusers.reset_index()
    # print("CORE USERS:\n", coreusers)
    return cos_coreusers

In [6]:
def CUR_ExtractCoreUsers(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    
    # print(userItemMatrix)

    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    # print("UNIQUE UID:", unique_uid)
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])

    userItemMatrix = np.zeros(shape=(u_len, m_len))
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[int(row['uid'])]][int(row['mid'])] = int(row['ratings'])
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    mat = userItemMatrix
    print("MAT:", mat)
    print(mat.shape)
    # C, c_ind = select_cols(mat, int(u_len * percent_core_user)) ## getting 20% core users
    r, r_ind= select_rows(mat, int(u_len * percent_core_user))
    # print("r", r)
    # print("r_ind len", len(r_ind))

    len_r_ind = len(r_ind)
    new_r_ind = [rev_uid_dict[r_ind[i]] for i in range(len_r_ind)]
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(new_r_ind))]
    # # coreusers.reset_index()
    # # print("CORE USERS:\n", coreusers)
    return cur_coreusers

In [7]:
def RandomSampling(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)

    how_much_to_sample = int(percent_core_user * u_len)
    indices = random.sample(range(u_len), how_much_to_sample)
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(indices))]
    return cur_coreusers

In [8]:
class largest_leveragescores_Sampler:
    def __init__(self, A, k, Q,N):
        """ Create largest k-leverage scores Sampler for the matrix :math:`A` for k-low rank apparoximation.
        :param A: 
            Matrix :math:`A`.
        :type A: 
            array_type
        :param Q: 
            Matrix containig the right singular vectors of :math:`A`.
        :type Q: 
            array_type
        :param k: 
            The order of low rank apparoximation.
        :type k: 
            int
        :param N: 
            The dimension of subsampling (the number of columns) of A.
        :type N: 
            int
        """
        self.A = A
        # print("k is", type(k))
        self.Q = np.transpose(Q[0:k,:])
        self.N = N
        self.k = k
        
        self.sampling_list = []
        self.lvs_array = self.Estimate_Leverage_Scores()
    def Estimate_Leverage_Scores(self):
        return 1/(self.k)*np.diag(np.dot(self.Q,self.Q.T))
    def MultiRounds(self):
        temp_list = list(reversed(np.argsort(self.lvs_array)))
        sampled_indices_ = temp_list[0:self.k]
        self.sampling_list = sampled_indices_
        return sampled_indices_
from scipy.sparse.linalg import svds
def Extract_LLS(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    
    # print(userItemMatrix)

    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    # print("UNIQUE UID:", unique_uid)
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])

    userItemMatrix = np.zeros(shape=(u_len, m_len))
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[int(row['uid'])]][int(row['mid'])] = int(row['ratings'])
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)
    A = userItemMatrix.T
    number_of_coreusers = int(u_len * percent_core_user)
    print(number_of_coreusers)
    u4, s4, Q = svds(A, k=number_of_coreusers)
    lls = largest_leveragescores_Sampler(A, number_of_coreusers, Q, number_of_coreusers)
    indx = lls.MultiRounds()
    # indx = indx.
    print("LENGTH OF INDEX:", len(indx))
    print("INDICES ARE:", indx)
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(indx))]
    # # coreusers.reset_index()
    # # print("CORE USERS:\n", coreusers)
    return cur_coreusers

In [9]:
core_users = Extract_LLS(df, len(np.unique(df['uid'])), unique_mids, percent_core_user=0.40)
support_user_list = np.unique(core_users['uid'])
print("CORE USERS:" ,core_users)
print("NUMBER OF CORE USERS:", len(support_user_list))

USER LEN: 2212
MOVIE LEN: 95872
USER ITEM MATRIX: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
884
LENGTH OF INDEX: 884
INDICES ARE: [787, 1137, 1235, 1255, 1194, 894, 2056, 114, 1205, 1642, 2143, 1565, 901, 1643, 1249, 2198, 1462, 337, 340, 1667, 1694, 1419, 1192, 929, 1107, 955, 1509, 1987, 1911, 2209, 1003, 2003, 1821, 1832, 2019, 2036, 402, 2007, 1297, 1520, 1405, 1864, 977, 1254, 1329, 1082, 1191, 881, 322, 2059, 1931, 1915, 1611, 1919, 1264, 2181, 1482, 1121, 1539, 1498, 1564, 986, 1665, 1523, 2177, 971, 807, 308, 1360, 23, 2034, 2186, 2182, 1866, 976, 2130, 1234, 427, 1538, 1727, 1112, 1617, 1836, 411, 328, 1305, 1026, 338, 1540, 1949, 731, 1209, 429, 1804, 1926, 1469, 2125, 1651, 1318, 1282, 350, 1017, 1826, 1869, 1313, 920, 1456, 1296, 2119, 2121, 2066, 866, 1150, 306, 1202, 1166, 2030, 441, 1546, 2200, 2075, 1048, 1421, 1324, 1991, 1167, 1423, 1788, 904, 931, 1741,

### CODE SNIPPET FOR GETTING NESTED CORE USERS:

In [10]:
core_users_nested = CUR_ExtractCoreUsers(core_users, len(np.unique(core_users['uid'])), unique_mids, percent_core_user=0.50)
print(len(np.unique(core_users_nested['uid'])))
print("NESTED CORE USERS:\n" ,core_users_nested)

USER LEN: 884
MOVIE LEN: 95872
USER ITEM MATRIX: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
MAT: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(884, 95872)
[0.00044968 0.00049554 0.00038351 0.00035704 0.00055433 0.00044506
 0.00234227 0.00149462 0.00213944 0.00094614 0.00055894 0.00066821
 0.00053186 0.00050293 0.00058942 0.00046753 0.00072731 0.00045922
 0.00063343 0.00062266 0.0013318  0.00038135 0.00048077 0.00042813
 0.00118006 0.00058942 0.00067498 0.0081007  0.00045153 0.00032133
 0.0003081  0.00062389 0.00083349 0.00029025 0.00087966 0.00152602
 0.00042198 0.00070422 0.00039674 0.00321424 0.0002964  0.00038997
 0.00029363 0.00076362 0.00033795 0.00063435 0.00294062 0.00145523
 0.00266607 0.00048754 0.00041921 0.00073839 0.00146661 0.00274517
 0.00054263 0.00

#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [11]:
core_users_index_list_nested = core_users_nested.index.to_list()
non_core_user_index = (df.index.difference(core_users_nested.index))
non_core_user_index = non_core_user_index.tolist()

core_users_df_nested = df.loc[core_users_index_list_nested]
non_core_user_df = df.loc[non_core_user_index]
print("NON CORE USERS:" ,non_core_user_df)
print("CORE USERS:", core_users_nested)
print(len(np.unique(non_core_user_df['uid'])))
print(np.unique(non_core_user_df['uid']))
li = [1]*len(non_core_user_df)
new_non_core_user_df = pd.DataFrame({'uid':non_core_user_df['uid'].tolist(), 'mid':non_core_user_df['mid'].tolist(), 'ratings':li})
print("NEW DF:", new_non_core_user_df)

NON CORE USERS:          uid    mid  ratings
285     1707  53320        1
286     1707  60332        5
287     1707  56218        5
288     1707  56068        4
289     1707  51775        4
...      ...    ...      ...
227246  1814   4305        5
227247  1814   5877        5
227248  1814    809        5
227249  1814   1022        4
227250  1814   1051        5

[87073 rows x 3 columns]
CORE USERS:          uid    mid  ratings
0        613  55945        3
1        613  81196        4
2        613  32933        4
3        613  62950        5
4        613  59289        4
...      ...    ...      ...
226685  1348  78309        3
226686  1348  83113        2
226687  1348  80709        5
226688  1348  17170        3
226689  1348   6066        5

[140178 rows x 3 columns]
1770
[   0    1    2 ... 2208 2210 2211]
NEW DF:         uid    mid  ratings
0      1707  53320        1
1      1707  60332        1
2      1707  56218        1
3      1707  56068        1
4      1707  51775        1
...   

In [12]:
# core_users_df.to_csv("ml-1m_core_users.csv", index=False, header=False)
new_non_core_user_df.to_csv("douban_new_non_core_users.csv", index=False, header=False)
print(len(np.unique(new_non_core_user_df['uid'])))
print(len(np.unique(new_non_core_user_df['mid'])))

1770
43522


In [13]:
uid_of_core_users = np.unique(core_users_df_nested['uid']).tolist()
uid_non_core_users = np.unique(non_core_user_df['uid']).tolist()
import pickle
with open("douban_core_users_and_non_core_users_obj.pkl", "wb") as f:
    pickle.dump(uid_of_core_users, f)
    pickle.dump(uid_non_core_users, f)
    pickle.dump(non_core_user_df,f)

In [14]:
matrix = np.zeros((unique_uids, unique_mids))
for index, row in non_core_user_df.iterrows():
    matrix[int(row['uid'])][int(row['mid'])] = int(1)
## write this numpy matrix to a file
with open("douban_non_core_user_numpy_matrix.pkl", "wb") as f:
    pickle.dump(matrix, f)
    pickle.dump(unique_uids, f)
    pickle.dump(unique_mids, f)

In [15]:
support_test_df = core_users_df_nested.groupby("uid").tail(1)
# print(len(df_gt_30))
support_train_df = core_users_df_nested.drop(core_users_df_nested.groupby('uid').tail(1).index, inplace=False)
# print("#TEST INSTANCES: " ,len(support_test_df))
# print("#TRAIN INSTANCES: " ,len(support_train_df))
assert(len(core_users_df_nested) == len(support_test_df) + len(support_train_df))
# print(len(test_df))
# print(len(train_df))
query_test_df = non_core_user_df.groupby("uid").tail(1)
query_train_df = non_core_user_df.drop(non_core_user_df.groupby('uid').tail(1).index, inplace=False)
assert(len(non_core_user_df)== len(query_test_df) + len(query_train_df))

In [16]:
train_ui_dic = {}    
for user in range(unique_uids):
    train_ui_dic[user] = []
for index,row in support_train_df.iterrows():
        train_ui_dic[row['uid']].append(row['mid'])

#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [17]:
unique_uids_in_support_trian = np.unique(np.array(core_users_df_nested['uid']))
unique_uids_in_query_trian = np.unique(query_train_df['uid'])
print(len(unique_uids_in_support_trian))
support_test_df = support_test_df.loc[support_test_df['uid'].isin(unique_uids_in_support_trian)]
print("SUPPORT TEST DF:" ,support_test_df)
query_test_df = query_test_df
print("QUERY TEST DF:\n", query_test_df)

442
SUPPORT TEST DF:          uid    mid  ratings
284      613   6169        4
485     1639  25721        3
576     1977  78976        5
760      478  79963        4
1420    1264  18677        2
...      ...    ...      ...
224411   930   3806        5
224870  1766  16754        2
225896  1617  81268        5
226241  1740  72519        5
226689  1348   6066        5

[442 rows x 3 columns]
QUERY TEST DF:
          uid    mid  ratings
295     1707  87421        5
299     1620  18452        4
302     1889  74757        4
307      447  75026        4
343     2010  81065        4
...      ...    ...      ...
226705  1960   5762        4
226748  1219    187        5
226841   837  43146        1
226930  1067  94845        5
227250  1814   1051        5

[1770 rows x 3 columns]


#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [18]:
support_train = []
for index,row in core_users_df_nested.iterrows():
    support_train.append([row['uid'], row['mid'], row['ratings']])
query_train = []
for index, row in query_train_df.iterrows():
    query_train.append([row['uid'], row['mid'], row['ratings']])
support_test = []
for index, row in support_test_df.iterrows():
    support_test.append([row['uid'], row['mid'], row['ratings']])
query_test = []
for index, row in query_test_df.iterrows():
    query_test.append([row['uid'], row['mid'], row['ratings']])
user_his_dic = {}
for u in train_ui_dic.keys():
    user_his_dic[u] = train_ui_dic[u]
user_supp_list = np.unique(core_users_df_nested['uid']).tolist()

In [19]:
import pickle
with open("douban_input_for_getting_query_embeddings.pkl", "wb") as f:
    pickle.dump(support_train, f)
    pickle.dump(query_train, f)
    pickle.dump(support_test, f)
    pickle.dump(query_test, f)
    pickle.dump(user_supp_list, f)
    pickle.dump(user_his_dic, f)

generating non core user embeddings

In [20]:
!python pretrain-1m.py
!python train-1m.py
# !python test-1m.py

-------Dataset Info--------
split way [threshold] with threshold 30 training_ratio 1.0
train set size: support/query 140178/85303
test set size: support/query 442/1770
Epoch 0 Step 131: Train 4.9505 Reg: 0.6656
Test: 1.0248 MAE: 0.8670 RMSE: 1.0123
Val: 0.7608 MAE: 0.6879 RMSE: 0.8723
Epoch 1 Step 262: Train 0.7518 Reg: 0.6438
Test: 0.8767 MAE: 0.7910 RMSE: 0.9363
Val: 0.7149 MAE: 0.6695 RMSE: 0.8455
Epoch 2 Step 393: Train 0.7090 Reg: 0.9199
Test: 0.8109 MAE: 0.7603 RMSE: 0.9005
Val: 0.7002 MAE: 0.6668 RMSE: 0.8368
Epoch 3 Step 524: Train 0.6759 Reg: 1.1599
Test: 0.7526 MAE: 0.7363 RMSE: 0.8675
Val: 0.7008 MAE: 0.6649 RMSE: 0.8372
Epoch 4 Step 655: Train 0.6423 Reg: 1.3241
Test: 0.7137 MAE: 0.7110 RMSE: 0.8448
Val: 0.7020 MAE: 0.6666 RMSE: 0.8379
Epoch 5 Step 786: Train 0.6057 Reg: 1.4574
Test: 0.6848 MAE: 0.6831 RMSE: 0.8275
Val: 0.7103 MAE: 0.6696 RMSE: 0.8428
Epoch 6 Step 917: Train 0.5686 Reg: 1.5358
Test: 0.6357 MAE: 0.6582 RMSE: 0.7973
Val: 0.7176 MAE: 0.6717 RMSE: 0.8471
Epoch 

In [24]:
!python test-1m.py

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
Extra : False
-------Dataset Info--------
split way [threshold] with threshold 30 training_ratio 1.0
train set size: support/query 140178/85303
test set size: support/query 442/1770
CORE IS SELECTED:
SIZE OF MODEL: torch.Size([2212, 32])
SIZE OF MODEL: torch.Size([95872, 32])
torch.Size([2212, 95872])
MATRIX IS: tensor([[-9.8008e-10, -4.7807e-09, -2.2821e-07,  ...,  1.7130e-07,
          2.7591e-08,  3.8432e-08],
        [-1.1211e-08, -2.8426e-09, -4.0827e-06,  ...,  6.7347e-06,
          1.8111e-10,  8.7163e-07],
        [-1.9119e-09,  1.6175e-08,  8.2559e-07,  ..., -1.4087e-06,
         -3.1535e-08, -1.8115e-07],
        ...,
        [ 2.8586e-05, -5.8327e-05,  1.5924e-03,  ..., -2.2678e-03,
          3.6967e-05, -2.6993e-04],
        [-2.0551e-09,  2.2095e-09, -1.4603e-06,  ...,  2.2979e-06,
          2.5325e-08,  2.7943e-07],
        [-4.2709